# Analise Exploratória dos Dados

Fazer amostragem das bases

1. Identificar as variáveis críticas para qualidade de dados/ou considerar todas
2. Identificar quais documentos têm estrutura diferente e retornar id
3. Testar as hipóteses

## 1) Identificar as variáveis críticas para qualidade de dados/ou considerar todas

Utilizando o MongoDB Compass foi feita uma análise de esquema na qual foram identificadas:

campos principais
campos com mais campos null
campos indefinidos
A análise foi feita baseada em 1.000 documentos, 0,76% do total enviado.

A análise retornou um esquema da base.

In [83]:
import os
import json
import pandas as pd
import numpy as np

In [84]:
#estes são arquivos csv de frequência de campos, possuem amostras de 18 tribunais.
files = [it for it in os.listdir('../data/csv') if '.csv' in it]

In [85]:
#agrupa todos csv em um dataframe
df_stats = pd.DataFrame()
for it in files:
    df_stats = pd.concat([pd.read_csv(f'../data/csv/{it}'), df_stats], ignore_index=True, axis=0)
df_stats = df_stats.fillna(0)

In [105]:
#apresenta variáveis com mais valores faltantes, em ordem.
#entretanto nem todas são obrigatórias.
#(verificar quais são - MTD)
df.loc[:, df_stats.mean() < 1].mean().sort_values(ascending=False)

movimento_1.dataHora                               0.955357
dadosBasicos.assunto_1.principal                   0.928571
movimento_2.dataHora                               0.883929
movimento_3.dataHora                               0.821429
dadosBasicos.assunto_1.codigoNacional              0.767857
                                                     ...   
movimento_189.orgaoJulgador                        0.008929
movimento_189.orgaoJulgador.nomeOrgao              0.008929
movimento_189.orgaoJulgador.codigoMunicipioIBGE    0.008929
movimento_189.orgaoJulgador.codigoOrgao            0.008929
movimento_69.tipoDecisao                           0.008929
Length: 3692, dtype: float64

## 2) Identificar quais documentos têm estrutura diferente e retornar id



In [ ]:
# carregar os docs, selecionar as features anteriores, verificar quais tem campos nulos, retornar ids

In [76]:
# aqui carrega 25 json em 'data/processos-json' extrai conteúdo e põe num dataframe
global values
values = []
def json_frame(node, root=''):
    col = ''''''
    for k in node.keys():        
        col += f'{root}.{k}' if root else k
        col += ','
        t = type(node[k])
        if (t != list) & (t != dict): 
            values.append(node[k]) 
        else:
            values.append(True)
        if type(node[k]) == dict:
            col += json_frame(node[k], root=f'{root}.{k}' if root else k)
        if type(node[k]) == list:
            for i in range(len(node[k])):
                col += json_frame(node[k][i], root=f'{root}.{k}_{i+1}' if root else f'{k}_{i+1}')
    return col

data = pd.DataFrame()

#aqui itera documentos, lê nomes de campos e valores. no final cria frame com eles.
for json_path in get_paths(collection_path): 
    doc = load_json(collection_path + '/' + json_path)
    values = []
    cols = json_frame(doc)
    cols = cols.split(',')[0:-1]
    cols = dict(zip(cols, values))
    
    data = pd.concat([pd.DataFrame(data=[cols.values()], columns=cols.keys()), data])
data = data.fillna(np.nan)


In [92]:
# aqui seleciona variáveis com frequência maior que 0.6 (aleatoriamente escolhido)
# supõe-se que são importantes
features = df_stats.loc[:, df_stats.mean() > 0.6].columns

In [114]:
data.loc[:, features].isnull().sum().sort_values(ascending=False)

dadosBasicos.totalAssuntos                        8
movimento_3.movimentoNacional.codigoNacional      8
movimento_3.movimentoNacional                     8
movimento_8.dataHora                              7
movimento_7.dataHora                              7
movimento_6.dataHora                              7
movimento_2.movimentoNacional.codigoNacional      6
dadosBasicos.assunto_1.codigoNacional             6
movimento_5.dataHora                              6
movimento_2.movimentoNacional                     6
movimento_1.movimentoNacional.codigoNacional      5
movimento_1.movimentoNacional                     5
movimento_4.dataHora                              5
movimento_3.dataHora                              4
movimento_2.dataHora                              3
dadosBasicos.orgaoJulgador                        0
dadosBasicos                                      0
dadosBasicos.assunto                              0
dadosBasicos.assunto_1.principal                  0
dadosBasicos

## 3) Testar as hipóteses



In [56]:
# Existe um sistema predominante nos docs com dados importantes nulos
# O ano afeta a presença dos dados, processos mais antigos com mais dados faltantes

# Validando Campos

**Exemplos de inconsistências:**

- Classes e assuntos de matérias diferentes;
- Número do processo fora do padrão;
- Datas de movimento fora da ordem;
- Código da TPU do nível 1 ou 2;
- Processo sem classe, assunto ou movimento;
- Órgão julgador ou ramo da justiça com competência diferente da classe e assunto;
- Processos baixados sem julgamento.


In [97]:
data.describe(include='object')

,dadosBasicos.assunto_1.assuntoLocal,dadosBasicos.assunto_2.principal,dadosBasicos.assunto_2.assuntoLocal,dadosBasicos.numero,dadosBasicos.dataAjuizamento,dadosBasicos.orgaoJulgador.nomeOrgao,dadosBasicos.orgaoJulgador.codigoOrgao,dadosBasicos.orgaoJulgador.instancia,dadosBasicos.codigoLocalidade,siglaTribunal,...,movimento_182.identificadorMovimento,movimento_183.identificadorMovimento,movimento_184.identificadorMovimento,movimento_185.identificadorMovimento,movimento_186.identificadorMovimento,movimento_187.identificadorMovimento,dadosBasicos.assunto_1.descricao,dadosBasicos.assunto_2.descricao,movimento_5.complementoNacional_3.descricaoComplemento,movimento_5.complementoNacional_4.descricaoComplemento
count,5,6,1,25,25,25,25,25,25,25,...,1,1,1,1,1,1,1,1,1,1
unique,1,2,1,25,23,25,25,2,20,12,...,1,1,1,1,1,1,1,1,1,1
top,True,True,True,00210841819004058300,20200318000000,30ª ZONA ELEITORAL - SÃO BENTO DO SUL,14338,ORIG,3550308,TRF2,...,84990,84964,84941,84965,84997,84885,Execução Fiscal,Execução Fiscal,tipo_de_distribuicao_redistribuicao,tipo_de_distribuicao_redistribuicao
freq,5,3,1,1,3,1,1,22,2,3,...,1,1,1,1,1,1,1,1,1,1


### 1) Classes e assuntos de matérias diferentes:
obs.: esse acho que não pode ser alterado, pois não dá pra saber se é a classe ou assunto que está errado.

### 2) número do processo fora do padrão:
- padrão: NNNNNNN-DD.AAAA.J.TR.OOOO
obs.: este acho que não pode ser alterado. Mesmo que as informações que formam ele estiverem disponíveis no documento, ele já está correndo com número errado. 
Criar um alerta para que corrijam?

### 3) Datas de movimento fora da ordem:

### 4) Código da TPU do nível 1 ou 2:
obs.: talvez, se for pra preencher com os elementos pai, porque dá pra traçar o caminho de volta. O contrário não é possível.

In [ ]:
def corrige_nivel_tpu():
    pass

### 5) Processo sem classe, assunto ou movimento:
obs: não dá pra alterar. Mas impede de correr processo. Alerta?

In [ ]:
def get_processo_sem_classe_assunto_movimento():
    pass

### 6) Órgão julgador ou ramo da justiça com competência diferente da classe e assunto;
obs: não dá pra alterar. Mas é grave e pode atrasar o processo. De repente criar um alerta.

In [ ]:
def get_processo_orgao_competencia_diferente():
    pass

### 7) Processos baixados sem julgamento.

Código para extrair campos do json e criar tabela com frequência

In [4]:
collection_path = '../data/processos-json'

def get_paths(collection_path):
    docs = os.listdir(collection_path)
    docs = [it for it in docs if '.json' in it]
    return docs

def load_json(json_path):
    with open(json_path, 'r') as file:
        return json.load(file)[0]

def get_structure(node, root=''):
    struct = ''''''
    for k in node.keys():
        struct += f'{root}.{k}' if root else k
        struct += ','
        if type(node[k]) == dict:
            struct += get_structure(node[k], root=f'{root}.{k}' if root else k)
        if type(node[k]) == list:
            for i in range(len(node[k])):
                struct += get_structure(node[k][i], root=f'{root}.{k}_{i+1}' if root else f'{k}_{i+1}')
    return struct

In [ ]:
#retorna os campos com a frequência de cada um
df = pd.DataFrame()

count = 0
for json_path in get_paths(collection_path): 
    doc = load_json(collection_path + '/' + json_path)
    cols = get_structure(doc)
    cols = cols.split(',')[0:-1]
    for it in cols:
        df.loc[count, it] = 1
    count += 1
    
df = df.fillna(0)

In [ ]:
df.to_csv('features.csv', index=False)